<a href="https://colab.research.google.com/github/jmbost20/YouTransfer/blob/main/453_Group5_Cloned_Git_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup
!pip install transformers

#numpy==1.17.2
#torch==1.3.0
#transformers==2.1.1
##sklearn==0.0
#spacy==2.2.3
#nltk==3.4.5
#gensim==3.8.1
#tqdm==4.36.1

#Imports
import transformers
import sklearn
import spacy
import nltk
import gensim
import tqdm

import torch
from torch.utils.data import DataLoader
#from linguistic_style_transfer_pytorch.config import GeneralConfig, ModelConfig
#from linguistic_style_transfer_pytorch.data_loader import TextDataset
#from linguistic_style_transfer_pytorch.model import AdversarialVAE
from tqdm import tqdm, trange
import os
import numpy as np
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Git Clone
#!git clone https://github.com/h3lio5/linguistic-style-transfer-pytorch.git

In [ ]:
#@title Configs

#### THESE ARE CONFIG FUNCTION FROM THIS GITHUB (https://github.com/h3lio5/linguistic-style-transfer-pytorch)

import os

ROOT = '/content/linguistic-style-transfer-pytorch' #os.path.join(os.getcwd(),'linguistic_style_transfer_pytorch')


class GeneralConfig:
    """
    General configuration
    """

    def __init__(self):
        self.vocab_size = 10000
        self.bow_hidden_dim = 10000
        #self.data_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data")
        #self.vocab_save_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data")
        #self.train_pos_reviews_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "raw", "sentiment.train.1.txt")
        #self.train_neg_reviews_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "raw", "sentiment.train.0.txt")
        #self.train_text_file_path = os.path.join(
            #ROOT, "linguistic_style_transfer_pytorch", "data", "clean", "Corpus.npy") # **key** Changed
        self.train_text_file_path = '/content/drive/MyDrive/Style-Transfer/Data/Corpus.npy'
        #self.train_labels_file_path = os.path.join(
         #   ROOT, "linguistic_style_transfer_pytorch", "data", "clean", "labels.npy") # **key** Changed
        self.train_labels_file_path = '/content/drive/MyDrive/Style-Transfer/Data/labels.npy' 
        #self.pos_sentiment_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "lexicon", "positive-words.txt")
        #self.neg_sentiment_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "lexicon", "negative-words.txt")
        #self.word_embedding_text_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "embedding.txt")
        #self.word_embedding_path = os.path.join(
         #   ROOT, "linguistic_style_transfer_pytorch", "data", "word_embeddings.npy") # **key** Changed
        self.word_embedding_path = '/content/drive/MyDrive/Style-Transfer/Data/embedding_matrix.npy'
        #self.vocab_file_path = os.path.join(
        #    ROOT, "linguistic_style_transfer_pytorch", "data", "Vocabulary_obj.pth") # **key** Changed
        self.vocab_file_path = '/content/drive/MyDrive/Style-Transfer/Data/Vocabulary_obj.pth'
        #self.bow_file_path = os.path.join(
         #   ROOT, "linguistic_style_transfer_pytorch", "data", "BoW_file.json") # **key** Changed
        self.bow_file_path = '/content/drive/MyDrive/Style-Transfer/Data/BoW_file.json'
        #self.model_save_path = os.path.join(
         #   ROOT, "linguistic_style_transfer_pytorch", "checkpoints")
        self.model_save_path = '/content/drive/MyDrive/Style-Transfer/Models/checkpoints'
        self.avg_style_emb_path = '/content/drive/MyDrive/Style-Transfer/Models/checkpoints/avg_style_emb.pkl'
        self.embedding_size = 64
        self.pad_token = 0
        self.sos_token = 1
        self.unk_token = 2
        self.predefined_word_index = {
            "<pad>": 0,
            "<sos>": 1,
            "<unk>": 2,
        }
        self.filter_sentiment_words = True
        self.filter_stopwords = True


class ModelConfig:
    """
    Model configuration
    """

    def __init__(self):
        # vocab size after including special tokens
        self.vocab_size = 10000
        self.epochs = 20
        # batch setting
        self.batch_size = 128
        # layer sizes
        self.embedding_size = 64
        self.hidden_dim = 256
        self.style_hidden_dim = 8 # **key** Going to want to increase as we add more youtubers but keep smaller than content
        self.content_hidden_dim = 128
        # generative embedding dim = style_hidden_dim + content_hidden_dim
        self.generative_emb_dim = 136
        self.num_style = 2 # **key** Of youtubers we currently have. Fine at 2 for now.
        self.content_bow_dim = 10000
        # dropout
        self.dropout = 0.8
        # sequence length settings
        self.max_seq_len = 20 # originally 15, changed to 20 from 75
        # learning rates
        self.autoencoder_lr = 0.001
        self.style_adversary_lr = 0.001
        self.content_adversary_lr = 0.001
        # loss weights
        self.style_multitask_loss_weight = 10
        self.content_multitask_loss_weight = 3
        self.style_adversary_loss_weight = 1
        self.content_adversary_loss_weight = 0.03
        self.style_kl_lambda = 0.03
        self.content_kl_lambda = 0.03
        # kl annealing max iterations
        self.kl_anneal_iterations = 20000
        # noise
        self.epsilon = 1e-8
        self.label_smoothing = 0.1

In [ ]:
#@title Text Datasets

#### THIS IS TEXT DATASET FUNCTIONS FROM THIS GITHUB (https://github.com/h3lio5/linguistic-style-transfer-pytorch)

from torch.utils.data import Dataset
import json

gconfig = GeneralConfig()
mconfig = ModelConfig()


class TextDataset(Dataset):
    """
    Dataset for text data
    """

    def __init__(self, mode='train'):
        super(TextDataset, self).__init__()
        # load train data
        #with open(gconfig.train_text_file_path) as f:
        #    self.train_data = f.readlines()
        self.train_data = np.load(gconfig.train_text_file_path, allow_pickle = True)
        print(self.train_data.shape)
        # load train labels
        #with open(gconfig.train_labels_file_path) as f:
        #    self.train_labels = f.readlines()
        self.train_labels = np.load(gconfig.train_labels_file_path)
        # load vocab
        #with open(gconfig.vocab_file_path) as f:
        #    self.vocab = torch.load(f)
        self.vocab = torch.load(gconfig.vocab_file_path)
        # load bow vocab
        #with open(gconfig.bow_file_path) as f:
        #   self.bow_filtered_vocab_indices = json.load(f)
        self.bow_filtered_vocab_indices = self.vocab
        self.label2index = {'Kings and Generals': [0, 1], '3Blue1Brown': [1, 0]} # **key**  One hot encoded

    def _padding(self, token_ids):
        """
        Utility function to add padding to and trim the sentence
        """
        if len(token_ids) > mconfig.max_seq_len:
            return token_ids[:mconfig.max_seq_len]
        token_ids = token_ids + \
            (mconfig.max_seq_len-len(token_ids))*[gconfig.pad_token]

        return token_ids

    def _sentence_tokenid(self, sentence):
        """
        Returns token ids of individual words of the sentence
        """
        token_ids = [self.vocab[word] if self.vocab[word] != -1 else self.vocab['<unk>'] for word in sentence]
        padded_token_ids = self._padding(token_ids)
        return padded_token_ids, len(token_ids)

    def _get_bow_representations(self, text_sequence):
        """
        Returns BOW representation of every sequence of the batch
        """

        sequence_bow_representation = np.zeros(
            shape=gconfig.bow_hidden_dim, dtype=np.float32)
        # Iterate over each word in the sequence
        for index in text_sequence:
            if index in self.bow_filtered_vocab_indices:
                bow_index = self.bow_filtered_vocab_indices[index]
                sequence_bow_representation[bow_index] += 1
        sequence_bow_representation /= np.max(
            [np.sum(sequence_bow_representation), 1])

        return np.asarray(sequence_bow_representation)

    def __len__(self):
        """
        Returns the total number of samples
        """
        return len(self.train_labels)

    def __getitem__(self, index):
        """
        Returns:
            token_ids: token ids of the sentence
            seq_len : length of the sentence before padding
            label   : label of the sentence
            bow_rep : Bag of Words representation of the sentence
        """
        sentence = self.train_data[index]
        label = self.label2index[self.train_labels[index].strip()]
        token_ids, seq_len = self._sentence_tokenid(sentence)
        bow_rep = self._get_bow_representations(sentence)
        return (torch.LongTensor(token_ids), torch.LongTensor([seq_len]), torch.LongTensor(label), torch.FloatTensor(bow_rep))

In [ ]:
#@title Preprocess code
import re
import logging
#from linguistic_style_transfer_pytorch.config import GeneralConfig

config = GeneralConfig()


class Preprocessor():
    """
    Preprocessor class
    """

    def __init__(self):

        print("Preprocessor instantiated")

    def _clean_text(self, string):
        """
        Clean the raw text file
        """
        string = string.replace(".", "")
        string = string.replace(".", "")
        string = string.replace("\n", " ")
        string = string.replace(" 's", " is")
        string = string.replace("'m", " am")
        string = string.replace("'ve", " have")
        string = string.replace("n't", " not")
        string = string.replace("'re", " are")
        string = string.replace("'d", " would")
        string = string.replace("'ll", " will")
        string = string.replace("\r", " ")
        string = string.replace("\n", " ")
        string = re.sub(r'\d+', "number", string)
        string = ''.join(x for x in string if x.isalnum() or x == " ")
        string = re.sub(r'\s{2,}', " ", string)
        string = string.strip().lower()

        return string

    def preprocess(self):
        """
        Preprocesses the train text data 
        """
        print("Preprocessing started")
        with open(config.train_text_file_path, 'w') as text_file, open(config.train_labels_file_path, 'w') as labels_file:
            with open(config.train_pos_reviews_file_path, 'r') as reviews_file:
                for line in reviews_file:
                    line = self._clean_text(line)
                    if len(line) > 0:
                        text_file.write(line + "\n")
                        labels_file.write("pos" + "\n")
            with open(config.train_neg_reviews_file_path, 'r') as reviews_file:
                for line in reviews_file:
                    line = self._clean_text(line)
                    if len(line) > 0:
                        text_file.write(line + "\n")
                        labels_file.write("pos" + "\n")
        print("Processing complete ")


In [ ]:
#@title AdversarialVAE
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn
import math

mconfig = ModelConfig()
gconfig = GeneralConfig()


class AdversarialVAE(nn.Module):
    """
    Model architecture defined according to the paper
    'Disentangled Representation Learning for Non-Parallel Text Style Transfer'
    https://www.aclweb.org/anthology/P19-1041.pdf
    """

    def __init__(self, weight):
        """
        Initialize networks
        """
        super(AdversarialVAE, self).__init__()
        # word embeddings
        self.embedding = nn.Embedding.from_pretrained(weight)
        #================ Encoder model =============#
        self.encoder = nn.GRU(
            mconfig.embedding_size, mconfig.hidden_dim, batch_first=True, bidirectional=True)
        # content latent embedding
        self.content_mu = nn.Linear(
            2*mconfig.hidden_dim, mconfig.content_hidden_dim)  
        self.content_log_var = nn.Linear(
            2*mconfig.hidden_dim, mconfig.content_hidden_dim)
        # style latent embedding
        self.style_mu = nn.Linear(
            2*mconfig.hidden_dim, mconfig.style_hidden_dim)
        self.style_log_var = nn.Linear(
            2*mconfig.hidden_dim, mconfig.style_hidden_dim)
        #=============== Discriminator/adversary============#
        self.style_disc = nn.Linear(
            mconfig.content_hidden_dim, mconfig.num_style)
        self.content_disc = nn.Linear(
            mconfig.style_hidden_dim, mconfig.content_bow_dim)
        #=============== Classifier =============#
        self.content_classifier = nn.Linear(
            mconfig.content_hidden_dim, mconfig.content_bow_dim)
        self.style_classifier = nn.Linear(
            mconfig.style_hidden_dim, mconfig.num_style)
        #=============== Decoder =================#
        # Note: input embeddings are concatenated with the sampled latent vector at every step
        self.decoder = nn.GRUCell(
            mconfig.embedding_size + mconfig.generative_emb_dim, mconfig.hidden_dim)
        self.projector = nn.Linear(mconfig.hidden_dim, mconfig.vocab_size)
        #============== Average label embedding ======#
        # Used during inference to transfer style.
        # Each element of the dict consists of average of latent style embeddings
        # of all the sentences of that particular label/style.

        # **key** t0 -> negative, 1 -> positive
        # USE THIS TO CLASSIFY THE AUTHORS OF THE YOUTUBE VIDEOS
        self.avg_style_emb = {
            0: torch.zeros(mconfig.style_hidden_dim),
            1: torch.zeros(mconfig.style_hidden_dim)
        }
        # Used to maintain a running average
        self.num_neg_styles = 0
        self.num_pos_styles = 0
        # dropout
        self.dropout = nn.Dropout(mconfig.dropout)

    def forward(self, sequences, seq_lengths, style_labels, content_bow, iteration, last_epoch):
        """
        Args:
            sequences : token indices of input sentences of shape = (batch_size,max_seq_length)
            seq_lengths: actual lengths of input sentences before padding, shape = (batch_size,1)
            style_labels: labels of sentiment of the input sentences, shape = (batch_size,2)
            content_bow: Bag of Words representations of the input sentences, shape = (batch_size,bow_hidden_size)
            iteration: number of iterations completed till now; used for KL annealing
            last_epoch: save average style embeddings if last_epoch is true
        Returns:
            content_disc_loss: loss incurred by content discriminator/adversary
            style_disc_loss  : loss incurred by style discriminator/adversary
            vae_and_classifier_loss : consists of loss incurred by autoencoder, content and style
                                      classifiers
        """
        # pack the sequences to reduce unnecessary computations
        # It requires the sentences to be sorted in descending order to take
        # full advantage
        seq_lengths, perm_index = seq_lengths.sort(descending=True)
        sequences = sequences[perm_index]
        embedded_seqs = self.dropout(self.embedding(sequences))
        print("pre encoder embedded_seqs size: ", embedded_seqs.size())
        print("seq_lengths: ", seq_lengths.size())
        packed_seqs = pack_padded_sequence(
            embedded_seqs, lengths=seq_lengths, batch_first=True)
        print("packed seqs: ", packed_seqs.data.size())
        packed_output, (_) = self.encoder(packed_seqs)
        print("packed_output: ", packed_output.data.size())
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        print("output: ", output.size())
        sentence_emb = output[torch.arange(output.size(0)), seq_lengths-1]
        # get content and style embeddings from the sentence embeddings,i.e. final_hidden_state
        print("after encoder sentence_emb: ", sentence_emb.size())
        content_emb_mu, content_emb_log_var = self.get_content_emb(
            sentence_emb)
        style_emb_mu, style_emb_log_var = self.get_style_emb(
            sentence_emb)
        # sample content and style embeddings from their respective latent spaces
        sampled_content_emb = self.sample_prior(
            content_emb_mu, content_emb_log_var)
        sampled_style_emb = self.sample_prior(
            style_emb_mu, style_emb_log_var)
        # Generative embedding
        generative_emb = torch.cat(
            (sampled_style_emb, sampled_content_emb), axis=1)
        # Update the average style embeddings for different styles
        # This will be used in transfering the style of a sentence
        # during inference
        
        #if last_epoch:
        #print("pre")
        self.update_average_style_emb(sampled_style_emb, style_labels)
        #print("post")
        #print("now: ", self.avg_style_emb)

        #=========== Losses on content space =============#
        # Discriminator Loss
        content_disc_preds = self.get_content_disc_preds(sampled_style_emb)
        content_disc_loss = self.get_content_disc_loss(
            content_disc_preds, content_bow)
        # adversarial entropy
        content_entropy_loss = self.get_entropy_loss(content_disc_preds)
        # Multitask loss
        content_mul_loss = self.get_content_mul_loss(
            sampled_content_emb, content_bow)

        #============ Losses on style space ================#
        # Discriminator loss
        style_disc_preds = self.get_style_disc_preds(sampled_content_emb)
        style_disc_loss = self.get_style_disc_loss(
            style_disc_preds, style_labels)
        # adversarial entropy
        style_entropy_loss = self.get_entropy_loss(style_disc_preds)
        # Multitask loss
        style_mul_loss = self.get_style_mul_loss(
            sampled_style_emb, style_labels)

        #============== KL losses ===========#
        # Style space
        style_kl_loss = self.get_kl_loss(
            style_emb_mu, style_emb_log_var)
        if iteration < mconfig.kl_anneal_iterations:
            style_kl_loss = self.get_annealed_weight(
                iteration, mconfig.style_kl_lambda) * style_kl_loss
        # Content space
        content_kl_loss = self.get_kl_loss(
            content_emb_mu, content_emb_log_var)
        if iteration < mconfig.kl_anneal_iterations:
            content_kl_loss = self.get_annealed_weight(
                iteration, mconfig.content_kl_lambda) * content_kl_loss

        #=============== reconstruction ================#
        reconstructed_sentences = self.generate_sentences(
            sequences, generative_emb)
        reconstruction_loss = self.get_recon_loss(
            reconstructed_sentences, sequences)
        #================ total weighted loss ==========#
        vae_and_classifier_loss = mconfig.content_adversary_loss_weight * content_entropy_loss + \
            mconfig.style_adversary_loss_weight * style_entropy_loss + \
            mconfig.style_multitask_loss_weight * style_mul_loss + \
            mconfig.content_multitask_loss_weight * content_mul_loss + \
            reconstruction_loss + style_kl_loss + content_kl_loss

        return content_disc_loss, style_disc_loss, vae_and_classifier_loss

    def get_style_content_emb(self, sequences, seq_lengths, style_labels, content_bow):
        """
        Args:
            sequences : token indices of input sentences of shape = (batch_size,max_seq_length)
            seq_lengths: actual lengths of input sentences before padding, shape = (batch_size,1)
            style_labels: labels of sentiment of the input sentences, shape = (batch_size,2)
            content_bow: Bag of Words representations of the input sentences, shape = (batch_size,bow_hidden_size)
        Returns:
            sampled_content_emb: content embeddings sampled from the content latent space, shape=(batch_size,content_hid_dim)
            sampled_style_emb:   style embeddings sampled from the style latent space, shape=(batch_size,style_hid_dim)
        """
        with torch.no_grad():
            # pack the sequences to reduce unnecessary computations
            # It requires the sentences to be sorted in descending order to take
            # full advantage
            seq_lengths, perm_index = seq_lengths.sort(descending=True)
            sequences = sequences[perm_index]
            embedded_seqs = self.embedding(sequences)
            packed_seqs = pack_padded_sequence(
                embedded_seqs, lengths=seq_lengths, batch_first=True)
            packed_output, (_) = self.encoder(packed_seqs)
            output, _ = pad_packed_sequence(packed_output, batch_first=True)
            sentence_emb = output[torch.arange(output.size(0)), seq_lengths-1]
            # get content and style embeddings from the sentence embeddings,i.e. final_hidden_state
            content_emb_mu, content_emb_log_var = self.get_content_emb(
                sentence_emb)
            style_emb_mu, style_emb_log_var = self.get_style_emb(
                sentence_emb)
            # sample content and style embeddings from their respective latent spaces
            sampled_content_emb = self.sample_prior(
                content_emb_mu, content_emb_log_var)
            sampled_style_emb = self.sample_prior(
                style_emb_mu, style_emb_log_var)

        return sampled_content_emb, sampled_style_emb

    def get_params(self):
        """
        Returns:
            content_disc_params: parameters of the content discriminator/adversary
            style_disc_params  : parameters of the style discriminator/adversary
            other_params       : parameters of the vae and classifiers
        """

        content_disc_params = self.content_disc.parameters()
        style_disc_params = self.style_disc.parameters()
        other_params = list(self.encoder.parameters()) + list(self.decoder.parameters()) + \
            list(self.style_classifier.parameters()) + \
            list(self.content_classifier.parameters())

        return content_disc_params, style_disc_params, other_params

    def get_content_emb(self, sentence_emb):
        """
        Args:
            sentence_emb: sentence embeddings of all the sentences in the batch, shape=(batch_size,2*gru_hidden_dim)
        Returns:
            mu: embedding of the mean of the Gaussian distribution of the content's latent space
            log_var: embedding of the log of variance of the Gaussian distribution of the content's latent space
        """
        #print(sentence_emb)
        mu = self.content_mu(sentence_emb)
        log_var = self.content_log_var(sentence_emb)

        return mu, log_var

    def get_style_emb(self, sentence_emb):
        """
        Args:
            sentence_emb: sentence embeddings of all the sentences in the batch, shape=(batch_size,2*gru_hidden_dim)
        Returns:
            mu: embedding of the mean of the Gaussian distribution of the style's latent space
            log_var: embedding of the log of variance of the Gaussian distribution of the style's latent space
        """
        #print(sentence_emb.size())
        mu = self.style_mu(sentence_emb)
        log_var = self.style_log_var(sentence_emb)

        return mu, log_var

    def sample_prior(self, mu, log_var):
        """
        Returns samples drawn from the latent space constrained to
        follow diagonal Gaussian
        """
        print("Mu size: ", mu.size())
        print("log_var size: ", log_var.size())
        epsilon = torch.randn(mu.size(-1),device=mu.device)
        #print("epsilon size: ", epsilon.size())
        return mu + epsilon*torch.exp(log_var)

    def update_average_style_emb(self, style_emb, style_labels):
        """
        Args:
            style_emb: batch of sampled style embeddings of the input sentences,shape = (batch_size,mconfig.style_hidden_dim)
            style_labels: style labels of the corresponding input sentences,shape = (batch_size,2)
        """
        # **key** will need to update to get average for every youtuber rather than just positive and negative
        neg_style_label = torch.LongTensor([0, 1], device=style_emb.device)
        # Iterate over the style labels
        for idx, label in enumerate(style_labels):
            # Calculate average for negative style
            if neg_style_label.equal(label):
                # Increment the counter for negative styles
                self.num_neg_styles = self.num_neg_styles + 1
                #print('author A increased count to:', self.num_neg_styles)
                # Calculate a running average of the negative style embedding
                self.avg_style_emb[0] = (
                    (self.num_neg_styles-1) * self.avg_style_emb[0] + style_emb[idx])/self.num_neg_styles
                #print('avg self embed author A:', self.avg_style_emb[0])
            else:
                # Increment the counter for positive styles
                self.num_pos_styles = self.num_pos_styles + 1
                # Calculate a running average of the positive style embedding
                self.avg_style_emb[1] = (
                    (self.num_pos_styles-1) * self.avg_style_emb[1] + style_emb[idx])/self.num_pos_styles
        #print(self.avg_style_emb)

    def get_content_disc_preds(self, style_emb):
        """
        Returns predictions about the content using style embedding
        as input
        output shape : [batch_size,content_bow_dim]
        """
        # predictions
        # Note: detach the style embedding since when don't want the gradient to flow
        #       all the way to the encoder. content_disc_loss is used only to change the
        #       parameters of the discriminator network
        preds = nn.Softmax(dim=1)(self.content_disc(
            self.dropout(style_emb.detach())))

        return preds

    def get_content_disc_loss(self, content_disc_preds, content_bow):
        """
        It essentially quantifies the amount of information about content
        contained in the style space
        Returns:
        cross entropy loss of content discriminator
        """
        # label smoothing
        smoothed_content_bow = content_bow * \
            (1-mconfig.label_smoothing) + \
            mconfig.label_smoothing/mconfig.content_bow_dim
        # calculate cross entropy loss
        content_disc_loss = nn.BCELoss()(content_disc_preds, smoothed_content_bow)

        return content_disc_loss

    def get_style_disc_preds(self, content_emb):
        """
        Returns predictions about style using content embeddings
        as input
        output shape: [batch_size,num_style]
        """
        # predictions
        # Note: detach the content embedding since when don't want the gradient to flow
        #       all the way to the encoder. style_disc_loss is used only to change the
        #       parameters of the discriminator network
        preds = nn.Softmax(dim=1)(self.style_disc(
            self.dropout(content_emb.detach())))

        return preds

    def get_style_disc_loss(self, style_disc_preds, style_labels):
        """
        It essentially quantifies the amount of information about style
        contained in the content space
        Returns:
        cross entropy loss of style discriminator
        """
        # label smoothing
        smoothed_style_labels = style_labels * \
            (1-mconfig.label_smoothing) + \
            mconfig.label_smoothing/mconfig.num_style
        # calculate cross entropy loss

        style_disc_loss = nn.BCELoss()(style_disc_preds, smoothed_style_labels)

        return style_disc_loss

    def get_entropy_loss(self, preds):
        """
        Returns the entropy loss: negative of the entropy present in the
        input distribution
        """
        return torch.mean(torch.sum(preds * torch.log(preds + mconfig.epsilon), dim=1))

    def get_content_mul_loss(self, content_emb, content_bow):
        """
        This loss quantifies the amount of content information preserved
        in the content space
        Returns:
        cross entropy loss of the content classifier
        """
        # predictions
        preds = nn.Softmax(dim=1)(
            self.content_classifier(self.dropout(content_emb)))
        # label smoothing
        smoothed_content_bow = content_bow * \
            (1-mconfig.label_smoothing) + \
            mconfig.label_smoothing/mconfig.content_bow_dim
        # calculate cross entropy loss
        content_mul_loss = nn.BCELoss()(preds, smoothed_content_bow)

        return content_mul_loss

    def get_style_mul_loss(self, style_emb, style_labels):
        """
        This loss quantifies the amount of style information preserved
        in the style space
        Returns:
        cross entropy loss of the style classifier
        """
        # predictions
        preds = nn.Softmax(dim=1)(
            self.style_classifier(self.dropout(style_emb)))
        # label smoothing
        smoothed_style_labels = style_labels * \
            (1-mconfig.label_smoothing) + \
            mconfig.label_smoothing/mconfig.num_style
        # calculate cross entropy loss
        style_mul_loss = nn.BCELoss()(preds, smoothed_style_labels)

        return style_mul_loss

    def get_annealed_weight(self, iteration, lambda_weight):
        """
        Args:
            iteration(int): Number of iterations compeleted till now
            lambda_weight(float): KL penalty weight
        Returns:
            Annealed weight(float)
        """
        return (math.tanh(
            (iteration - mconfig.kl_anneal_iterations * 1.5) /
            (mconfig.kl_anneal_iterations / 3))
            + 1) * lambda_weight

    def get_kl_loss(self, mu, log_var):
        """
        Args:
            mu: batch of means of the gaussian distribution followed by the latent variables
            log_var: batch of log variances(log_var) of the gaussian distribution followed by the latent variables
        Returns:
            total loss(float)
        """
        kl_loss = torch.mean((-0.5*torch.sum(1+log_var -
                                             log_var.exp()-mu.pow(2), dim=1)))
        return kl_loss

    def generate_sentences(self, input_sentences, latent_emb, inference=False):
        """
        Args:
            latent_emb: generative embedding formed by the concatenation of sampled style and
                       content latent embeddings, shape = (batch_size,mconfig.generative_emb_dim)
            input_sentences: batch of token indices of input sentences, shape = (batch_size,max_seq_length)
                            It is of type 'None' when the function is called in inference mode
            inference: bool indicating whether train/inference mode
        Returns:
            output_sentences: batch of token indices or logits of generated sentences based on the
            mode of operation.
            modes:
                train: shape = (max_seq_len,batch_size,vocab_size)
                inference: shape = (max_seq_len,batch_size)
        """
        # Training mode
        if not inference:
            # Prepend the input sentences with <sos> token
            sos_token_tensor = torch.LongTensor(
                [gconfig.predefined_word_index['<sos>']], device=input_sentences.device).unsqueeze(0).repeat(mconfig.batch_size, 1)
            #print(sos_token_tensor.size(), 'sos token tensor')
            #print(input_sentences.size(), 'input sentence size')
            input_sentences = torch.cat(
                (sos_token_tensor, input_sentences), dim=1)
            #print("input sentences: ", input_sentences.size())
            sentence_embs = self.dropout(self.embedding(input_sentences))
            #print("sentence embs: ", sentence_embs.size())
            # Make the latent embedding compatible for concatenation
            # by repeating it for max_seq_len + 1(additional one bcoz <sos> tokens were added)
            #print("latent_emb pre: ", latent_emb.size())
            latent_emb = latent_emb.unsqueeze(1).repeat(
                1, mconfig.max_seq_len+1, 1)
            #print("latent_emb post: ", latent_emb.size())
            gen_sent_embs = torch.cat(
                (sentence_embs, latent_emb), dim=2)
            # Delete latent embedding and sos token tensor to reduce memory usage
            del latent_emb, sos_token_tensor
            output_sentences = torch.zeros(
                mconfig.max_seq_len, mconfig.batch_size, mconfig.vocab_size, device=input_sentences.device)
            # initialize hidden state
            hidden_states = torch.zeros(
                mconfig.batch_size, mconfig.hidden_dim, device=input_sentences.device)
            # generate sentences one word at a time in a loop
            for idx in range(mconfig.max_seq_len):
                # get words at the index idx from all the batches
                words = gen_sent_embs[:, idx, :]
                hidden_states = self.decoder(words, hidden_states)
                # project over vocab space
                next_word_logits = self.projector(hidden_states)
                output_sentences[idx] = next_word_logits
        # if inference mode is on
        else:

            sos_token_tensor = torch.LongTensor(
                [gconfig.predefined_word_index['<sos>']], device=latent_emb.device) #.unsqueeze(0)
            #print("sos_token_tensor : ", sos_token_tensor)
            #print("sos_token_tensor unsqueezed: ", sos_token_tensor.unsqueeze(0))
            word_emb = self.embedding(sos_token_tensor)
            #print('unsqueezed embedding - ',self.embedding(sos_token_tensor.unsqueeze(0)))
            #print('word-embedding:', word_emb)
            #print(word_emb.size())
            #print('latentembedding:', latent_emb)
            #print(latent_emb.size())

            hidden_states = torch.zeros(
                1, mconfig.hidden_dim, device=latent_emb.device)
            #print('hidden state:',hidden_states)
            #print(hidden_states.size())
            # Store output sentences
            output_sentences = torch.zeros(
                mconfig.max_seq_len, 1, device=latent_emb.device)
            #print("output_sentence: ", output_sentences)
            
            #end_sentence_embedding = word_emb.repeat(mconfig.max_seq_length, 1)
            with torch.no_grad():
                # Greedily generate new words at a time 
                if mconfig.max_seq_len > latent_emb.size()[0]:
                  max_index = latent_emb.size()[0]
                else:
                  max_index = mconfig.max_seq_len
                for idx in range(max_index):
                  #print("idx", idx)
                  #print("word emb size: ", word_emb.size())
                  #print("latent emb size: ", latent_emb.size())
                  #print("hidden_states size: ", hidden_states.size())
                  #print("word embed", word_emb.size())
                  #print("Latent embed", latent_emb.size())
                  #word_and_latent_emb = torch.cat((end_sentence_embedding[1:idx, :], latent_emb[1:idx, :]), axis=1)
                  #word_and_latent_emb = torch.cat((word_emb, latent_emb.mean(axis=0).view(1, -1)), axis=1)
                  #print('word_emb  pre cat - ', word_emb)
                  #print('word_emb size pre cat - ', word_emb.size())
                  #print('latent_emb  pre cat - ', latent_emb)
                  #print('latent_emb size  pre cat - ', latent_emb.size())
                  #print('latent_emb[idx, :]  pre cat - ', latent_emb[idx,:].view(1, -1))
                  word_and_latent_emb = torch.cat((word_emb, latent_emb[idx, :].view(1, -1)), axis=1)
                  #print("word_and_latent_emb_grand size: ", word_and_latent_emb.size())
                  hidden_states = self.decoder(word_and_latent_emb, hidden_states)   #Can we concatenate tensors to remember sequences?
                  #print("hidden_states: ", hidden_states)
                  #print("hidden_states size: ", hidden_states.size())
                  #print("hidden states: ", hidden_states)
                  next_word_probs = nn.Softmax(dim=1)(
                      self.projector(hidden_states))
                  #print('next_word probs', next_word_probs)
                  #print('original format', next_word_probs.argmax(1))

                  values, indices = torch.topk(next_word_probs,20, dim=1)
                  #print("indices - ", indices)

                  next_word = torch.tensor([indices.tolist()[0][2]], dtype=torch.int64) #next_word_probs.argmax(1)
                  #print('new next word alt method',next_word)
                  #print(vocab.lookup_token(next_word))
                  #print(indices.tolist()[0][1])

                  #values, indices = torch.topk(next_word_probs,2)
                  #p = torch.no_grad(indices[1])
                  #print("test p", indices)
                  #print('print 2nd most likely', next_word_probs.argmax(2))
                  #output_sentences[idx] = next_word
                  #word_emb = self.embedding(next_word)
                  #print('output_sentences:', output_sentences)

                  #next_word = next_word_probs.argmax(1)
                  output_sentences[idx] = next_word
                  word_emb = self.embedding(next_word)
                  

        return output_sentences

    def get_recon_loss(self, output_logits, input_sentences):
        """
        Args:
            output_logits: logits of output sentences at each time step, shape = (max_seq_length,batch_size,vocab_size)
            input_sentences: batch of token indices of input sentences, shape = (batch_size,max_seq_length)
        Returns:
            reconstruction loss calculated using cross entropy loss function
        """

        loss = nn.CrossEntropyLoss(ignore_index=0)
        recon_loss = loss(
            output_logits.view(-1, mconfig.vocab_size), input_sentences.view(-1))

        return recon_loss

    def transfer_style(self, sequence, style):
        """
        Args:
            sequence : token indices of input sentence of shape = (random_seq_length,) 
            style: target style
        Returns:
            transfered_sentence: token indices of style transfered sentence, shape=(random_seq_length,)
        """
        #print("Sequence: ", sequence)
        #print("Style: ", style)

        # pack the sequences to reduce unnecessary computations
        # It requires the sentences to be sorted in descending order to take
        # full advantage
        seq_len = torch.tensor(len(sequence)).view(1)
        seq_len = seq_len.cpu()
        print("seq_len: ", seq_len)
        #print("output: ", output.size())
        #sentence_emb = output[torch.arange(output.size(0)), seq_lengths-1]
        temp = [index if index != -1 else torch.tensor(vocab.vocab['<unk>']) for index in sequence] #fix unk token
        #print(temp)
        #[temp.append(torch.tensor(vocab.vocab['<pad>'])) for i in range(20 - len(temp))]  #add padding
        temp = torch.tensor(temp) 
        temp = temp.cpu()
        embedded_seq = self.embedding(temp)
        embedded_seq = embedded_seq.view(1, len(sequence), -1)
        #print('embedding size: ', embedded_seq.size())
        print('embedding sequence size: ', embedded_seq.size())
        #print(seq_len.size())

        packed_seqs = pack_padded_sequence(
            embedded_seq, lengths=seq_len, batch_first=True)
        print('packed_output: ', packed_seqs.data.size()) 
        packed_output, (_) = self.encoder(packed_seqs)
        print('packed_output: ', packed_output.data.size()) 
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        #output, final_hidden_state = self.encoder(embedded_seq)
        print('output: ', output.size()) # Needs to be 1 by (size sentence) by 512
        print(output.size(0))
        print(seq_len-1)
        sentence_emb = output[torch.arange(output.size(0)), seq_len-1]
        print("sentence_embedding: ", sentence_emb.size())
        #print(sentence_emb)
        # get content embeddings
        # Note that we need not calculate style embeddings since we
        # use the target style embedding
        content_emb_mu, content_emb_log_var = self.get_content_emb(
            sentence_emb)
        #print("content_emb_mu: ", content_emb_mu.size())
        #print("content_emb_log_var: ", content_emb_log_var.size())

        # sample content embeddings latent space
        sampled_content_emb = self.sample_prior(
            content_emb_mu, content_emb_log_var)
        #print('Content Embed Mu - ', content_emb_mu)
        print('Sampled Content Emb: ', sampled_content_emb.size())


        # Get the approximate estimate of the target style embedding
        target_style_emb = self.avg_style_emb[style]
        #print('Target Style Emb - ', target_style_emb)
        print("Target Style Embedding: ")
        #print(target_style_emb)
        print(target_style_emb.size())
        x = target_style_emb.repeat(sampled_content_emb.size()[0], 1)
        print("X: ")
        #print(x)
        print(x.size())

        # Generative embedding
        #print("Sampled Content Embedding: ")
        #print(sampled_content_emb)
        #print(sampled_content_emb.size())

        #chosen_sampled_content_emb = sampled_content_emb[1, :]
        #print('Chosen Sample Content Emb - ', chosen_sampled_content_emb)
        #print("Chosen_sampled_content_emb: ", chosen_sampled_content_emb)
        #print(chosen_sampled_content_emb.size())
        generative_emb = torch.cat(
            (x, sampled_content_emb), axis=1)
        #print('Generative Emb: ', generative_emb)
        print("Content embedding: ", sampled_content_emb.size())
        
        # Generate the style transfered sentences
        #transfered_sentence = self.generate_sentences(
        #    input_sentencs=None, latent_emb=generative_emb, inference=True)
        transfered_sentence = self.generate_sentences(
            input_sentences=None, latent_emb=generative_emb, inference=True)

        return transfered_sentence.view(-1)

Train

In [ ]:
#@title Train
import torch
from torch.utils.data import DataLoader

#from linguistic_style_transfer_pytorch.config import GeneralConfig, ModelConfig
#from linguistic_style_transfer_pytorch.data_loader import TextDataset
#from linguistic_style_transfer_pytorch.model import AdversarialVAE

from tqdm import tqdm, trange
import os
import numpy as np
import pickle

use_cuda = True if torch.cuda.is_available() else False


if __name__ == "__main__":

    mconfig = ModelConfig()
    gconfig = GeneralConfig()
    weights = torch.FloatTensor(np.load('/content/drive/MyDrive/Style-Transfer/Data/embedding_matrix.npy')) #/content/linguistic-style-transfer-pytorch/linguistic_style_transfer_pytorch/data/word_embeddings.npy
    
    model = AdversarialVAE(weight=weights)
    #checkpoints = torch.load('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/model_epoch_7.pt')  #last epoch trained
    #model.load_state_dict(checkpoints)
    #with open('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/avg_style_emb.pkl', 'rb') as f:
      #avg_style_embeddings = pickle.load(f)
    #model.avg_style_emb = avg_style_embeddings

    if use_cuda:
        model = model.cuda()

    #=============== Define dataloader ================#
    train_dataset = TextDataset(mode='train')
    train_dataloader = DataLoader(train_dataset, batch_size=mconfig.batch_size)
    content_discriminator_params, style_discriminator_params, vae_and_classifier_params = model.get_params()
    #============== Define optimizers ================#
    # content discriminator/adversary optimizer
    content_disc_opt = torch.optim.RMSprop(
        content_discriminator_params, lr=mconfig.content_adversary_lr)
    # style discriminaot/adversary optimizer
    style_disc_opt = torch.optim.RMSprop(
        style_discriminator_params, lr=mconfig.style_adversary_lr)
    # autoencoder and classifiers optimizer
    vae_and_cls_opt = torch.optim.Adam(
        vae_and_classifier_params, lr=mconfig.autoencoder_lr)
    print("Training started!")
    for epoch in trange(mconfig.epochs, desc="Epoch"):
        count = 0
        pass_count = 0
        for iteration, batch in enumerate(tqdm(train_dataloader)):
            count += 1
            #print(count)
            # unpacking
            if count == 344: break
            sequences, seq_lens, labels, bow_rep = batch
            if use_cuda:
              sequences = sequences.cuda()
              seq_lens = seq_lens.cuda()
              labels = labels.cuda()
              bow_rep = bow_rep.cuda()
            #####
            #print("\n\n\nSequences size: ", sequences.size())
            #print("Seq_lens: ", seq_lens.size())
            #print("Seq_lens: ", seq_lens.squeeze(1).size())
            #print("Labels: ", labels.size())
            #print("Bow representation: ", bow_rep.size(), "\n\n\n")
            #####
            content_disc_loss, style_disc_loss, vae_and_cls_loss = model(
                sequences, seq_lens.squeeze(1), labels, bow_rep, iteration+1, epoch == mconfig.epochs-1)

            #============== Update Adversary/Discriminator parameters ===========#
            # update content discriminator parametes
            # we need to retain the computation graph so that discriminator predictions are
            # not freed as we need them to calculate entropy.
            # Note that even detaching the discriminator branch won't help us since it
            # will be freed and delete all the intermediary values(predictions, in our case).
            # Hence, with no access to this branch we can't backprop the entropy loss
            content_disc_loss.backward(retain_graph=True)
            content_disc_opt.step()
            content_disc_opt.zero_grad()

            # update style discriminator parameters
            style_disc_loss.backward(retain_graph=True)
            style_disc_opt.step()
            style_disc_opt.zero_grad()

            #=============== Update VAE and classifier parameters ===============#
            vae_and_cls_loss.backward()
            vae_and_cls_opt.step()
            vae_and_cls_opt.zero_grad()

        print("Saving states")
        #================ Saving states ==========================#
        if not os.path.exists(gconfig.model_save_path):
            os.mkdir(gconfig.model_save_path)
        # save model state
        torch.save(model.state_dict(), gconfig.model_save_path +
                   f'/model_epoch_{epoch + 1}.pt')
        # save optimizers states
        torch.save({'content_disc': content_disc_opt.state_dict(
        ), 'style_disc': style_disc_opt.state_dict(), 'vae_and_cls': vae_and_cls_opt.state_dict()}, gconfig.model_save_path+f'/opt_epoch_{epoch + 1}.pt')
        # Save approximate estimate of different style embeddings after the last epoch
        pickle.dump(model.avg_style_emb, open(gconfig.avg_style_emb_path, 'wb'))
        break
    print("Training completed!!!")
  

(44120, 20)
Training started!


  0%|          | 0/885 [00:00<?, ?it/s]

pre encoder embedded_seqs size:  torch.Size([128, 20, 64])
seq_lengths:  torch.Size([128])
packed seqs:  torch.Size([2560, 64])
packed_output:  torch.Size([2560, 512])
output:  torch.Size([128, 20, 512])
after encoder sentence_emb:  torch.Size([128, 512])
Mu size:  torch.Size([128, 128])
log_var size:  torch.Size([128, 128])
Mu size:  torch.Size([128, 8])
log_var size:  torch.Size([128, 8])
input sentences:  torch.Size([128, 21])
latent_emb pre:  torch.Size([128, 136])
latent_emb post:  torch.Size([128, 21, 136])


Epoch:   0%|          | 0/20 [00:02<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
#@title Open Previous Models
#Load from Style-Transfer/Models/checkpoints
import torch
model_checkpoint = torch.load('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/model_epoch_6.pt') # This is the actual AdversarialVAE model

# We should probably change the name of end models after we go all the way through -- name them by their date of creation

In [ ]:
#@title Generate
import torch
import os
import argparse
import numpy as np
import pickle
import sys
#sys.path.append('/content/linguistic-style-transfer-pytorch/')
#from config import GeneralConfig
#from model import AdversarialVAE

gconfig = GeneralConfig()

# load word embeddings
weights = torch.FloatTensor(np.load('/content/drive/MyDrive/Style-Transfer/Data/embedding_matrix.npy'))
# load checkpoint
path_load_checkpoints = ''
model_checkpoint = torch.load('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/model_epoch_6.pt') # This is the actual AdversarialVAE model
# Load model
model = AdversarialVAE(weight=weights)
model.load_state_dict(model_checkpoint)
model.eval()
# Load average style embeddings
with open('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/avg_style_emb.pkl', 'rb') as f:
    avg_style_embeddings = pickle.load(f)

#avg_style_embeddings = pickle.load('/content/drive/MyDrive/Style-Transfer/Models/checkpoints/avg_style_emb.pkl')
# set avg_style_emb attribute of the model
model.avg_style_emb = avg_style_embeddings
# load vocab
vocab = torch.load(gconfig.vocab_file_path)
label2index = {'Kings and Generals': 0, '3Blue1Brown': 1} # **key** will need to change to the youtubers


# Read input sentence
#source_sentence = input("Enter the source sentence\n")
#target_style = input("Enter the target style: Kings and Generals or 3Blue1Brown\n")  
source_sentence =  '<sos> the integral of the function is a sum of infinite distances' #'test test hello '#"kings and generals channel make videos about this topic"  #'Try this sentence now, I bet its really hard'  
target_style_id = 1

# Get token ids
token_ids = [vocab[word] for word in source_sentence.split()]
token_ids = torch.LongTensor(token_ids)
#target_style_id = torch.LongTensor(label2index[target_style])
# Get transfered sentence token ids
#print(token_ids)
target_tokenids = model.transfer_style(token_ids, target_style_id)
print(target_tokenids)
target_sentence = " ".join([vocab.lookup_token(idx) for idx in target_tokenids])
print("Style transfered sentence: \n\n{}".format(target_sentence))


seq_len:  tensor([12])
embedding sequence size:  torch.Size([1, 12, 64])
packed_output:  torch.Size([12, 64])
packed_output:  torch.Size([12, 512])
output:  torch.Size([1, 12, 512])
1
tensor([11])
sentence_embedding:  torch.Size([1, 512])
Mu size:  torch.Size([1, 128])
log_var size:  torch.Size([1, 128])
Sampled Content Emb:  torch.Size([1, 128])
Target Style Embedding: 
torch.Size([8])
X: 
torch.Size([1, 8])
Content embedding:  torch.Size([1, 128])
tensor([5898.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.])
Style transfered sentence: 

center. <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


In [ ]:
<pad> <sos> reducing acts 25 brains , saints saints velocity velocity 2/16th sick kütahı continuously resentment helped rapid harsh etc

In [ ]:
<pad> <sos> reducing acts , 25 saints unchallenged velocity brains resentment 2/16th continuously creature 4,000 kütahı kütahı etc sick independently

In [ ]:
word = ''
#a = [word + '<pad>' for i in range(20)]

for i in range(20):
  word += ' supporting'

In [ ]:
word

' supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting supporting'

In [ ]:
m_prop = 4/17

In [ ]:
7*60

420

In [ ]:
m_prop*420 #same as ben, ideally ben has little more to talk about model

98.82352941176471

In [ ]:
jo_prop = 3/17
jo_prop*420

74.11764705882354

In [ ]:
ja_prop = 6/17
ja_prop*420

148.23529411764707